In [ ]:
#initalised libraries
import pandas as pd
import numpy as np
import re
import regex
import spacy

#additional libraries to output a spreadsheet at the end
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Border, Side, PatternFill, Font
from openpyxl.utils import get_column_letter

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

#access to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load

from google.colab import drive
drive.mount("/content/drive")
path = "/content/drive/MyDrive/ICS5110 Project/Final Code & Report"
%cd {path}
#local_news_articles = pd.read_csv("/content/drive/MyDrive/ICS5110 Project/local_news_articles.csv")
#local_news_articles = pd.read_csv("/content/drive/Shared with me/ICS5110 Project/local_news_articles.csv")
local_news_articles = pd.read_csv("local_news_articles.csv")
#police_press_releases = pd.read_csv("/content/drive/Shared with me/ICS5110 Project/police_press_releases.csv")
#police_press_releases = pd.read_csv("/content/drive/MyDrive/ICS5110 Project/police_press_releases.csv")
police_press_releases = pd.read_csv("police_press_releases.csv")

# Add a 'source' column to each DataFrame
local_news_articles['source'] = 'news_article'
police_press_releases['source'] = 'police_report'

# combining the two csv files
source = pd.concat([police_press_releases['source'], local_news_articles['source']], axis=0)
title = pd.concat([police_press_releases['title'], local_news_articles['title']], axis=0)
published_date = pd.concat([police_press_releases['date_published'], local_news_articles['publish_date']], axis=0)
content_details = pd.concat([police_press_releases['content'], local_news_articles['content']], axis=0)

csv_data = {'source': source, 'published_date': published_date, 'title': title,'content': content_details}
combined_traffic_data = pd.DataFrame(csv_data).reset_index(drop=True)

#display(combined_traffic_data.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ICS5110 Project/Final Code & Report


In [ ]:
#initalising new columns in the dataframe
combined_traffic_data["hospital flag"] = pd.NA
combined_traffic_data["date of incident"] = pd.NA
combined_traffic_data["weekday of incident"] = pd.NA
combined_traffic_data["time of incident"] = pd.NA

#changing datatype to datetime
#combined_traffic_data["published_date"] = pd.to_datetime(combined_traffic_data["published_date"])
combined_traffic_data["published_date"] = pd.to_datetime(combined_traffic_data["published_date"], format='mixed', dayfirst=True)

In [ ]:
# Hospital Flag -> true if the hospital names are featured in the content, false otherwise
combined_traffic_data["hospital flag"] = combined_traffic_data["content"].str.contains(r"mater dei|gozo general", case=False, na=False)

In [ ]:
#Intalising a dictionary witih the days of the weeks
weekday_map = { "monday": 0, "tuesday": 1, "wednesday": 2,"thursday": 3, "friday": 4, "saturday": 5, "sunday": 6}
weekday_names = {v: k.capitalize() for k, v in weekday_map.items()}

# Creating regex rule to find any mention of a weekday anywhere in the content (case-insensitive)
pattern_weekday = re.compile(r"\b(monday|tuesday|wednesday|thursday|friday|saturday|sunday)\b",re.IGNORECASE)

# Incident Date Extraction
# Rule 1) Extracting incident date based on mentions of "Today" "This Morning" "Yesterday" "Last Night" in the content

#Creating regex rule to find any of "Today" or "This morning"
is_today = combined_traffic_data["content"].str.contains(r"\b(?:today|this morning)\b", case=False, na=False)

#Creating regex rule to find any of "Yesterday" or "Last Night"
is_yesterday = combined_traffic_data["content"].str.contains(r"\b(?:yesterday|last night)\b", case=False, na=False)

#Applying regex rules to content
mask_today_fill = is_today & combined_traffic_data["date of incident"].isna()
mask_yest_fill = is_yesterday & combined_traffic_data["date of incident"].isna()

#if a match on "Today" or "This Morning" is found published_date = incident_date
combined_traffic_data.loc[mask_today_fill, "date of incident"] = (combined_traffic_data.loc[mask_today_fill, "published_date"])

#if a match on "TYesterday" or "Last Night" is found published_date - 1 day = incident_date
combined_traffic_data.loc[mask_yest_fill, "date of incident"] = (combined_traffic_data.loc[mask_yest_fill, "published_date"] - pd.Timedelta(days=1))

# Rule 2) Determine Weekday of Incident from Date of Incident

# identifying the weekday from the identified date of the incident
dates_dt = pd.to_datetime(combined_traffic_data["date of incident"], errors="coerce")
mask_infer_weekday = (combined_traffic_data["weekday of incident"].isna() & dates_dt.notna())

combined_traffic_data.loc[mask_infer_weekday, "weekday of incident"] = (dates_dt[mask_infer_weekday].dt.weekday.map(weekday_names))

# Rule 3) Extract the weekday from content only when BOTH date incident + weekday of incident are blank
# applying regex rule to find mention of a weekday in entries where no date of incident was identified

combined_traffic_data["weekday_mentioned"] = (combined_traffic_data["content"].str.extract(pattern_weekday, expand=False).str.lower())

mask_fill_weekday_from_content = (combined_traffic_data["date of incident"].isna() & combined_traffic_data["weekday of incident"].isna() & combined_traffic_data["weekday_mentioned"].notna())
combined_traffic_data.loc[mask_fill_weekday_from_content, "weekday of incident"] = (combined_traffic_data.loc[mask_fill_weekday_from_content, "weekday_mentioned"].str.capitalize())

# Rule 4) If date of incident is blank but a weekday of incident is identified in rule 3, infer date

#vector storing weekday num - from dictionary of each respective published date
published_wd = combined_traffic_data["published_date"].dt.weekday

#vector storing weekday num - from dictionary of each respective weekday extracted from content
mentioned_wd_num = combined_traffic_data["weekday of incident"].astype(str).str.lower().map(weekday_map)

#difference in weekday number
days_back = (published_wd - mentioned_wd_num) % 7

#mask to assess which entries have no date of incident extracted thus far
mask_fill_date_from_weekday = (combined_traffic_data["date of incident"].isna()& mentioned_wd_num.notna())

#blank date of incidents are updated by shifting from published date depending on the number stored in days_back
combined_traffic_data.loc[mask_fill_date_from_weekday, "date of incident"] = (combined_traffic_data.loc[mask_fill_date_from_weekday, "published_date"]- pd.to_timedelta(days_back[mask_fill_date_from_weekday], unit="D"))

# Rule 5) For Police Reports : if still blank - published_date = incident_date
#creating a mask to determine which entries are policie reports and have no incident date identified
mask_police_fallback_date = ((combined_traffic_data["source"].astype(str).str.lower() == "police_report") & combined_traffic_data["date of incident"].isna())

#applying mask and setting incident_date = published_date
combined_traffic_data.loc[mask_police_fallback_date, "date of incident"] = (combined_traffic_data.loc[mask_police_fallback_date, "published_date"])

#re-run rule 2 to update weekday of incident for newly extracted dates of incident
combined_traffic_data["date of incident"] = pd.to_datetime(combined_traffic_data["date of incident"], errors="coerce").dt.date
dates_dt = pd.to_datetime(combined_traffic_data["date of incident"], errors="coerce")
mask_infer_weekday = (combined_traffic_data["weekday of incident"].isna() & dates_dt.notna())
combined_traffic_data.loc[mask_infer_weekday, "weekday of incident"] = (dates_dt[mask_infer_weekday].dt.weekday.map(weekday_names))


In [ ]:
# Extracting the time of accident in "0930hrs" format

# creating and applying regex pattern for time extraction (mainly for police reports) which is in the form HHMM
pattern_time = re.compile(r"([0-9:.]+)\s*hrs\b", re.IGNORECASE)
combined_traffic_data["time of incident"] = combined_traffic_data["content"].str.extract(pattern_time, expand=False)

# Handle textual time reference: noon or midday = 1200
combined_traffic_data.loc[combined_traffic_data["content"].str.contains(r"\bnoon|midday\b", case=False, na=False), "time of incident"] = "1200"

#creating regex pattern to match times like 4pm, 4 pm, 4.45pm, 4:45 p.m., 04:45 a.m., etc. + will ignore any times that follow the word updated (safeguard for newsarticles with updates)
pattern_time_ampm = re.compile(r"(?i)\b(?:updated\s+)?(?P<time>\d{1,2}(?:[.:]\d{2})?\s*(?:a\.?m\.?|p\.?m\.?|am|pm))\b")

#defining method to apply regex pattern across content and ignore the article update times
def extract_first_non_updated_time(text: str):
    if not isinstance(text, str):
        return pd.NA

    # Find all candidate matches
    matches = list(pattern_time_ampm.finditer(text))
    if not matches:
        return pd.NA

    # Keep only those NOT preceded by "updated " immediately before the time
    for m in matches:
        time_str = m.group("time")

        # Look at up to 8 chars before the time (length of "updated ")
        start = m.start("time")
        prefix = text[max(0, start - 10):start].lower()

        # If it ends with "updated " (optionally multiple spaces), skip it
        if re.search(r"updated\s*$", prefix, flags=re.IGNORECASE):
            continue

        return time_str.strip()

    return pd.NA

#applying regex pattern
combined_traffic_data["time_ampm"] = combined_traffic_data["content"].apply(extract_first_non_updated_time)

#defining method to normalise extracted times to HHMM format
def normalise_ampm_to_hhmm(t):
    if not isinstance(t, str):
        return pd.NA

    t = t.lower().strip()

    # Match times like 4pm, 4.45pm, 4:45 p.m., 04:45am
    m = re.match(r"(\d{1,2})(?:[.:](\d{2}))?\s*(a\.?m\.?|p\.?m\.?)", t)
    if not m:
        return pd.NA

    hour = int(m.group(1))
    minute = int(m.group(2)) if m.group(2) else 0
    period = m.group(3)

    if "p" in period and hour != 12:
        hour += 12
    if "a" in period and hour == 12:
        hour = 0

    return f"{hour:02d}{minute:02d}"

#maps times extracted and formatted to time vector in dataframe
mask_fill_time = (combined_traffic_data["time of incident"].isna()& combined_traffic_data["time_ampm"].notna())
combined_traffic_data.loc[mask_fill_time, "time of incident"] = (combined_traffic_data.loc[mask_fill_time, "time_ampm"].apply(normalise_ampm_to_hhmm))

def normalise_any_time_to_hhmm(t):
    if t is None or pd.isna(t):
        return pd.NA
    t = str(t).strip().lower()

    # Already 3-4 digits like 700 / 0930 / 1545
    digits = re.sub(r"[^\d]", "", t)
    if digits.isdigit():
        if len(digits) in (3, 4):
            return digits.zfill(4)

    # Try am/pm format again if something slipped through
    out = normalise_ampm_to_hhmm(t)
    return out if out is not pd.NA else pd.NA

combined_traffic_data["time of incident"] = combined_traffic_data["time of incident"].apply(normalise_any_time_to_hhmm)


In [ ]:
combined_traffic_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   source               432 non-null    object        
 1   published_date       432 non-null    datetime64[ns]
 2   title                432 non-null    object        
 3   content              432 non-null    object        
 4   hospital flag        432 non-null    bool          
 5   date of incident     379 non-null    object        
 6   weekday of incident  379 non-null    object        
 7   time of incident     300 non-null    object        
 8   weekday_mentioned    264 non-null    object        
 9   time_ampm            192 non-null    object        
dtypes: bool(1), datetime64[ns](1), object(8)
memory usage: 30.9+ KB


In [ ]:
combined_traffic_data.head()

,source,published_date,title,content,hospital flag,date of incident,weekday of incident,time of incident,weekday_mentioned,time_ampm
0,police_report,2025-10-09,Collision between a car and a motorbike in Żur...,"Today, at around 0930hrs, the Police were info...",True,2025-10-09,Thursday,0930,NaN,<NA>
1,police_report,2025-06-20,Car-motorcycle traffic accident,"Yesterday, at around 1830hrs, the Police were ...",True,2025-06-19,Thursday,1830,NaN,<NA>
2,police_report,2025-05-12,Car-motorcycle collision in Ħal Qormi,"Today, at around 0800hrs, the Police were info...",True,2025-05-12,Monday,0800,NaN,<NA>
3,police_report,2025-07-30,Collision between motorcycle and car in Għaxaq,"Yesterday, at around 1800hrs, the Police were ...",True,2025-07-29,Tuesday,1800,NaN,<NA>
4,police_report,2025-04-07,Car-motorcycle collision,"Yesterday, at around quarter to nine in the ev...",True,2025-04-06,Sunday,2045,NaN,<NA>


# Importing cities dataset


In [ ]:
# Importing dataset with cities

world_cities_dataset = pd.read_csv("worldcities.csv")
world_cities_dataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48060 entries, 0 to 48059
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city        48060 non-null  object 
 1   city_other  48058 non-null  object 
 2   lat         48060 non-null  float64
 3   lng         48060 non-null  float64
 4   country     48060 non-null  object 
 5   iso2        48027 non-null  object 
 6   iso3        48060 non-null  object 
 7   admin_name  47859 non-null  object 
 8   capital     15139 non-null  object 
 9   population  47810 non-null  float64
 10  id          48060 non-null  float64
dtypes: float64(4), object(7)
memory usage: 4.0+ MB


In [ ]:
# Removing useless columns

df_world_cities_reduced = world_cities_dataset.drop(columns = ["iso2", "iso3", "admin_name", "capital"])
df_world_cities_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48060 entries, 0 to 48059
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city        48060 non-null  object 
 1   city_other  48058 non-null  object 
 2   lat         48060 non-null  float64
 3   lng         48060 non-null  float64
 4   country     48060 non-null  object 
 5   population  47810 non-null  float64
 6   id          48060 non-null  float64
dtypes: float64(4), object(3)
memory usage: 2.6+ MB


In [ ]:
# Keeping only cities in Malta

df_malta_cities = df_world_cities_reduced[df_world_cities_reduced["country"] == "Malta"]
df_malta_cities.info()
df_malta_cities.head(50)

<class 'pandas.core.frame.DataFrame'>
Index: 69 entries, 1495 to 48059
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city        69 non-null     object 
 1   city_other  69 non-null     object 
 2   lat         69 non-null     float64
 3   lng         69 non-null     float64
 4   country     69 non-null     object 
 5   population  69 non-null     float64
 6   id          69 non-null     float64
dtypes: float64(4), object(3)
memory usage: 4.3+ KB


,city,city_other,lat,lng,country,population,id
1495,Valletta,Valletta,35.8983,14.5125,Malta,480134.0,1.470574e+09
6479,Qormi,Qormi,35.8794,14.4722,Malta,93897.0,1.470879e+09
16679,Saint Paul’s Bay,St Paul's Bay,35.9483,14.4017,Malta,32042.0,1.470880e+09
21120,Birkirkara,Birkirkara,35.8967,14.4625,Malta,24356.0,1.470754e+09
22474,Sliema,Sliema,35.9122,14.5042,Malta,22591.0,1.470230e+09
24520,Mosta,Mosta,35.9097,14.4261,Malta,20241.0,1.470177e+09
29933,Marsaskala,Marsascala,35.8625,14.5675,Malta,15579.0,1.470809e+09
30176,Żabbar,Zabbar,35.8772,14.5381,Malta,15404.0,1.470318e+09
30981,Naxxar,Naxxar,35.9150,14.4447,Malta,14890.0,1.470470e+09
31231,San Ġwann,St Julian's,35.9094,14.4786,Malta,14720.0,1.470073e+09


# Extracting City names from text

In [ ]:

# Creating helper functions

def sort_cities (cities_df, column_name):
  """
  Function: Returns the column with cities as a list starting form longest to shortest.

  Args:
    cities_df = Cities dataframe
    coulmn_name = column within cities_df that you want to extract and sort.

  Return: City names sorted from longest to shortest.
  """
  # If column name is not in the dataset, return an empty list.
  if column_name not in cities_df.columns:
    print(f"{column_name} is not in your dataset. Please check the name of the column")
    return []
  # Create sorted list. dropna(): removes any empty cells, tolist(): converts to python list, key = len: sorts by length, reverse = True: sorts from longest to shortest.
  cities_sorted = sorted(cities_df[column_name].dropna().tolist(), key = len, reverse = True)
  return cities_sorted


def search_for_city(text, city_list):
  """
  Function: Search for a matching city from the given list.

  Args:
      text: Input raw text which needs to be searched.
      city_list: List of cities to search for.

  Return: Found matching city (or none if there is no match)

  """

  # if text is empty, return noe=ne
  if pd.isna(text):
    return None

  # Convert text to lowercase.
  lowercase_text = text.lower()

# Normalise aphostrophies since not all are the same in the text - replace smart/curly quotes with regular apostrophes
  lowercase_text = lowercase_text.replace("’", "'")  # Curly apostrophe → regular
  lowercase_text = lowercase_text.replace("`", "'")  # Backtick → apostrophe

  # Location indicators - to improve chance of getting location of accident rather than location where driver/victim are from.
  # Adding \b before each word to make sure it only matches the individual word eg. doesnt match in from resid[in]g

  location_indicators = [
    r'\bon ', r'\bin ', r'\bat ', r'\bnear ', r'\balong ', r'\bbypass',
    r'\btriq ', r'\bdawret ', r'\broad ', r'\bstreet '
]

  found_city = None # Initialising found city variable

 # First pass - looking for cities which follow the pattern including location_indicators (high likelihood of being correct)
  for city in city_list:
    city_lower = city.lower() # Convert city name to lowercase

    # Loop through each of the indicator words till you find a match
    for indicator in location_indicators:
    # Create a search algorithm pattern
    # inidcator - each indicator in the location_idicators list (looped one by one)
    # (?:... This is a container for the words)
    # (?!from\b) This is a negative lookahead - checks that next word is not from (if it is, it fails immediately)
    # \w+\s+ - Matches a word followed by a space.
    # {0,5} - allows up to 5 words between the indicator and the city name
    # \b is a word boundary that makes sure a hit is only found if the whole word (bound by a space or punctuation is found.)
    # re.escape is a safety measure so that if a word has eg a ' it treats it as plain text and doesn't cause errors.
      pattern = indicator + r'(?:\w+\s+){0,5}?' + r'\b' + re.escape(city_lower) + r'\b'

      # Searches for the above pattern from the lowercase text.
      # Using re.finditer instead of re.search to check all instances of the city name eg. sometimes resident is from a city but accident also happens in the same city.
      for match in re.finditer(pattern,lowercase_text):

        # Checks the 20 chars before match for residence words
        start = max(0,match.start() - 20)
        text_before = lowercase_text[start:match.start()]

        # Skip if there are the following words
        if re.search(r'lives|resid|from', text_before):
          continue

        # Check the 10 characters after city end for road or street, eg Zabbar Road, Paola
        text_after = lowercase_text[match.end():match.end() + 10]
        if re.search(r'^\s*(road|street|bypass|avenue)', text_after):
          continue

        return city # Returns the original city name (not lowercase) if a match is found (otherwise it goes to the second pass)

  # Second pass: If nothing is found using the pattern above, do a broader search for the city name
  for city in city_list:
    city_lower = city.lower()

    pattern = r'\b' + re.escape(city_lower) + r'\b'

    for match in re.finditer(pattern,lowercase_text):
      # Checks the 20 chars before match for residence words
      start = max(0,match.start() - 20)
      text_before = lowercase_text[start:match.start()]

      # Skip if there are the following words
      if re.search(r'lives|resid|from', text_before):
        continue
              # Check the 10 characters after city end for road or street, eg Zabbar Road, Paola
      text_after = lowercase_text[match.end():match.end() + 10]
      if re.search(r'^\s*(road|street|bypass|avenue)', text_after):
        continue

      return city

  return None # Return none if no match is made in either pass

def get_city_info (cities_df,city_name, search_column):

  """
  Function: Matches the found city to the row in the database and extracts the info from that row.

  Args:
      cities_df: Dataset with city names and info
      city_name: Name of the match city (to look up)
      search_column: Name of the column in which to search for (city or city_other)

  Return: Dictionary with city name and it's relevant info.

  """
  # find matching city name
  city_info = cities_df[cities_df[search_column] == city_name].iloc[0]

  return {
      "id": city_info["id"],
      "city":city_info["city"],
      "latitude": city_info["lat"],
      "longitude": city_info["lng"],
      "population": city_info["population"]

  }

  # Main function

def extract_city_and_coordinates(text_df, cities_df):
  """
  Function: Extract city names from reports and add coordinates + population to them

  Args:
      text_df: Raw text
      cities_df: Cities dataframe

  Return: tuple with (matched_df, unmatched_df)

  """
# Using helper functions to sort city lists

  city_main_name = sort_cities(cities_df,"city")
  city_other_name = sort_cities(cities_df, "city_other")

  print(f"Searching through the {len(city_main_name)} Malta cities for matching locations.")

  # Initialize the list to store all rows
  matched_results = []


  # Loop to go through each report. It first searches the title, and then the content

  for idx, row in text_df.iterrows():
    title = row["title"]
    content = row["content"]

    # Create a dictionary for the current row
    current_result = row.to_dict()

    # Initialise the Variables
    found_city = None
    matched_from = None
    found_in = None


    # Step 1: Look for a match in the the title

    found_city = search_for_city(title, city_main_name)

    if found_city:
      matched_from = "city" # Holds from which column there was a match
      found_in = "title"

    if not found_city: # If no match, search the other city name column
      found_city = search_for_city(title, city_other_name)
      if found_city:
        matched_from = "city_other"
        found_in = "title"

      # Step 2: If there is still no match, look for a match in the content.

    if not found_city: # Looking for a match in main city name column
      found_city = search_for_city(content, city_main_name)
      if found_city:
        matched_from = "city" # Holds from which column there was a match
        found_in = "content"

    if not found_city: # If no match, search the other city name column
      found_city = search_for_city(content, city_other_name)
      if found_city:
        matched_from = "city_other"
        found_in = "content"

    if found_city:
            # Get the city data by calling the get_city_info function
      city_data = get_city_info(cities_df, found_city, matched_from)
      # Store matched result
      current_result.update({
          "city_id": city_data["id"],
          "city": city_data["city"],
          "found_in": found_in,
          "longitude": city_data["longitude"],
          "latitude": city_data["latitude"],
          "population": city_data["population"],

      })
    else:
      # if no matched resutls, store in unmatched_results dictionary
      print(f"No matched location found for row {idx}: '{row['title']}'")
      current_result.update({
          "city": None,
          "found_in": None,
          "longitude": None,
          "latitude": None,
          "population": None,
      })

  # Add current result to matched_results list
    matched_results.append(current_result)

 # Convert dictionary to dataframe
  matched_df = pd.DataFrame(matched_results)


   # Print summary
  print(f"\n{'='*80}")
  print(f"EXTRACTION SUMMARY")
  print(f"{'='*80}")


  print(f"Cities found: {matched_df['city'].notna().sum()}")
  print(f"Cities not found: {matched_df['city'].isna().sum()}")
  print(f"Success rate: {matched_df['city'].notna().sum() / len(matched_df) * 100:.1f}%")

  print("\nTop 10 cities found:")
  print(matched_df['city'].value_counts().head(10))

  return matched_df






In [ ]:
# ============================================================================
# Test on Combined dataset
# ============================================================================

print("\n" + "="*80)
print("TESTING ON Combined dataset")
print("="*80)

combined_with_cities_df = extract_city_and_coordinates(combined_traffic_data, df_malta_cities)

# Display results
#print("\nNews Articles - Sample Results:")
print(combined_with_cities_df.head(10))
combined_with_cities_df.to_excel('combined_with_cities.xlsx', index=False)
#print(news_unmatched_df[['title']].head(40))



TESTING ON Combined dataset
Searching through the 69 Malta cities for matching locations.
No matched location found for row 63: 'Cyclist grievously injured in San Gwann accident'
No matched location found for row 91: 'Man grievously injured in traffic accident'
No matched location found for row 113: 'Motorcyclist seriously hurt in St Paul's Bay bypass crash'
No matched location found for row 121: '‘I’m tired of going to friends’ funerals’: bikers' anger after skip crash death'
No matched location found for row 123: 'Doctors ask: Where is the Transport Safety Investigative Commission?'
No matched location found for row 127: 'Two motorcyclists injured in four-vehicle crash on Triq is-Salina'
No matched location found for row 129: 'Bus driver critically injured in crash with van on Regional Road'
No matched location found for row 130: 'Bus driver dies day after Regional Road crash'
No matched location found for row 137: 'Police issue 400 traffic charges and fines over Christmas period'
N

# Importing weather data for time and location of accident.



In [ ]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.7/684.7 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.3/144.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.1/394.1 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.8 MB/s eta 0:00:00


In [ ]:
# Basis of this code obtained from open-meteo API docs : https://open-meteo.com/en/docs/historical-weather-api

import openmeteo_requests

import requests_cache
import time
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Historical API
archive_url = "https://archive-api.open-meteo.com/v1/archive"

# List of chosen weather variables
daily_weather_variables = ["weather_code", "temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "precipitation_sum","precipitation_hours", "rain_sum", "wind_speed_10m_max"]

# Creating a copy of the df
locations_weather_df = combined_with_cities_df.copy()

# Creating a new temporary date string column for the API. If there is no date, it will put NaN
locations_weather_df["date_str"] = pd.to_datetime(locations_weather_df["date of incident"], errors="coerce").dt.strftime("%Y-%m-%d")

# initialising weather variables with weather in the name
for variable in daily_weather_variables:
	locations_weather_df[f"weather_{variable}"] = None

# Initialize 'skipped' counter
skipped = 0

# Looping through the entries in the df
for idx, row in locations_weather_df.iterrows():
	# Skip if coordinates or date are empty
	if pd.isna(row["date_str"]) or pd.isna(row["longitude"]) or pd.isna(row["latitude"]):
		skipped +=1
		continue

	# Extracting weather data according to the given variables.
	# Using try, except loop to avoid total crashes in case API fails.

	try:
		params = {
				 "latitude": row["latitude"],
				 "longitude": row["longitude"],
				 "start_date": row["date_str"],
				 "end_date": row["date_str"],
				 "daily": daily_weather_variables,
				 "timezone": "Europe/Berlin" }

		responses = openmeteo.weather_api(archive_url, params=params)
		daily = responses[0].Daily()

		# Loop for extracting the daily variables form open_meteo API.
		# order of extraction will be the same as they were listed in daily_weather_variables

		for i, var in enumerate(daily_weather_variables):
			values = daily.Variables(i).ValuesAsNumpy()

			# if the request is successful and returns a number larger than 0, add it to the df
			if len(values) >0:
				locations_weather_df.at[idx, f"weather_{var}"] = values[0] # The api returns an array of weather, we only need the first one.

	except Exception as error:
		print(f" Error in row {idx}: {error}")


# Converting weather codes to categories.
# Code conversion from https://gist.github.com/stellasphere/9490c195ed2b53c707087c8c2db4ec0c
def weather_categories(code):
	if pd.isna(code): return None
	code = int(code)
	if code in [0,1]: return "sunny/clear"
	if code in [2,3]: return "cloudy"
	if code in [45,48]: return "foggy"
	if code in [51, 53, 55, 61, 63, 65]: return "rain/drizzle"
	return "other"

# Adding a new column with the weather category
locations_weather_df["weather_category"] = locations_weather_df["weather_weather_code"].apply(weather_categories)



locations_weather_df.to_excel("locations_weather.xlsx")

combined_cities_weather_df = locations_weather_df.copy()

In [ ]:
combined_cities_weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   source                       432 non-null    object        
 1   published_date               432 non-null    datetime64[ns]
 2   title                        432 non-null    object        
 3   content                      432 non-null    object        
 4   hospital flag                432 non-null    bool          
 5   date of incident             379 non-null    object        
 6   weekday of incident          379 non-null    object        
 7   time of incident             300 non-null    object        
 8   weekday_mentioned            264 non-null    object        
 9   time_ampm                    192 non-null    object        
 10  city_id                      353 non-null    float64       
 11  city                         353 non-null    

In [ ]:
combined_cities_weather_df.head()

,source,published_date,title,content,hospital flag,date of incident,weekday of incident,time of incident,weekday_mentioned,time_ampm,...,date_str,weather_weather_code,weather_temperature_2m_max,weather_temperature_2m_min,weather_temperature_2m_mean,weather_precipitation_sum,weather_precipitation_hours,weather_rain_sum,weather_wind_speed_10m_max,weather_category
0,police_report,2025-10-09,Collision between a car and a motorbike in Żur...,"Today, at around 0930hrs, the Police were info...",True,2025-10-09,Thursday,0930,NaN,None,...,2025-10-09,3.0,23.651501,17.251501,20.405668,0.0,0.0,0.0,9.82279,cloudy
1,police_report,2025-06-20,Car-motorcycle traffic accident,"Yesterday, at around 1830hrs, the Police were ...",True,2025-06-19,Thursday,1830,NaN,None,...,2025-06-19,2.0,28.551001,22.451,25.332254,0.0,0.0,0.0,9.418661,cloudy
2,police_report,2025-05-12,Car-motorcycle collision in Ħal Qormi,"Today, at around 0800hrs, the Police were info...",True,2025-05-12,Monday,0800,NaN,None,...,2025-05-12,3.0,23.1,18.450001,20.070833,0.0,0.0,0.0,20.118975,cloudy
3,police_report,2025-07-30,Collision between motorcycle and car in Għaxaq,"Yesterday, at around 1800hrs, the Police were ...",True,2025-07-29,Tuesday,1800,NaN,None,...,2025-07-29,2.0,27.720501,23.670502,25.393417,0.0,0.0,0.0,31.644184,cloudy
4,police_report,2025-04-07,Car-motorcycle collision,"Yesterday, at around quarter to nine in the ev...",True,2025-04-06,Sunday,2045,NaN,None,...,2025-04-06,51.0,17.455502,11.0055,14.405502,0.4,4.0,0.4,18.57143,rain/drizzle


In [ ]:
combined_cities_weather_df.to_excel("combined_cities_weather.xlsx")

In [ ]:
preprocessed_data = combined_cities_weather_df

In [ ]:
res1 = list(preprocessed_data.columns)

In [ ]:
#function to check if any dictionary word appears in a sentence
def contains_dictionary_word(sentence,dictionary):
    sentence = sentence.lower()
    words = re.findall(r"\b\w+\b",sentence)
    return int(any(word in words for word in dictionary))

In [ ]:
#Dictionaries
car_dictionary = {"car","privatecar","passenger","saloon","hatchback","estate","sedan","coupe","convertible","cabriolet","suv","crossover","4x4","toyota",
"ford","volkswagen","vw","hyundai","kia","nissan","peugeot","renault","citroen","fiat","skoda","seat","opel","suzuki","mazda","honda","bmw","mercedes","mercedesbenz",
"audi","volvo","mini","alfa","romeo","lancia","smart","dacia","cupra","landrover","range","rover","jeep","tesla","mg","byd","polestar","mitsubishi","chevrolet","lexus","porsche",
"jaguar","car","cars","carmotorcycle","vancar","hitandrun","electric","twocar"} #ok
motorcycle_dictionary = {"motorcycle","motorbike","motorcyclist","biker","rider","powered twowheeler","scooter","moped","quad bike","quadbike","honda","yamaha","kymco","sym",
"piaggio","vespa", "suzuki","kawasaki","bmw","ducati","ktm","benelli","cfmoto","zontes","royal","enfield","husqvarna","aprilia","motoguzzi","moto","guzzi","gilera","scooters",
"scooter","keeway","malaguti","polaris","canam","motorist","motorists","motorcyclists","carmotorcycle","motorcycletruck","vanmotorcycle","escooter"} #ok
heavy_vehicle_dictionary = {"lorry","truck","articulated","lorry","trailer","semitrailer","heavy","hgv","commercial","delivery","panel",
"van","bus","route","coach","public","transport","publictransportvehicle","publictransport","vanmotorcycle","vancar","motorcycletruck"}#ok
pedestrian_dictionary = {"pedestrian","onfoot","foot","walker","crossing","pedestrians"} #ok
fatal_dictionary = {"dead", "death","fatal", "died","passed","away","succumbed","fatality","fatalities","life","lost","deceased"} #ok
injury_dictionary = {"injured","injuries"} #ok
hospital_dictionary = {"hospitalised","hospitalized","hospital","admitted","medical", "treatment", "stable"} #ok
severe_dictionary = {"grievously","injured","seriously","severe","critical","condition","serious","lifethreatening","intensive", "unstable","extensive",
"hurt","death","fatal","serious","injured","seriously","grievously","killed","dies","hospital","hospitalised","suffers","injuries"} #ok
minor_dictionary = {"minor","slight","light","discharged","superficial", "cuts","bruises"} #ok
accident_dictionary = {"crash","accident","collision","accidents","crashes","collission","impact"} #ok
bike_dictionary = {"cyclist"} #ok
male_gender_dictionary = {"man","men","male","him"} #ok
female_gender_dictionary = {"woman","women","female","her"} #ok
drug_alcohol_dictionary = {"alcohol","drunk","drug","drugs","arrested","prison"} #ok
traffic_dictionary = {"traffic"} #ok
control_dictionary = {"tree", "pole", "singlevehicle","wall","retaining","building","storefront","barrier","hit","sreike","offroad","skid","slip","lose","lost","roadside"}

In [ ]:
df = pd.DataFrame()

In [ ]:
#Using the title column
df["accident_flag"] = preprocessed_data["title"].apply(lambda s: contains_dictionary_word(s,accident_dictionary))
df["hospital_flag"] = preprocessed_data["title"].apply(lambda s: contains_dictionary_word(s,hospital_dictionary))
df["fatal_flag"] = preprocessed_data["title"].apply(lambda s: contains_dictionary_word(s,fatal_dictionary))
df["injury_flag"] = preprocessed_data["title"].apply(lambda s: contains_dictionary_word(s,injury_dictionary))
df["severe_flag"] = preprocessed_data["title"].apply(lambda s: contains_dictionary_word(s,severe_dictionary))
df["minor_flag"] = preprocessed_data["title"].apply(lambda s: contains_dictionary_word(s,minor_dictionary))
df["motorcycle_flag"] = preprocessed_data["title"].apply(lambda s: contains_dictionary_word(s,motorcycle_dictionary))
df["bike_flag"] = preprocessed_data["title"].apply(lambda s: contains_dictionary_word(s,bike_dictionary))
df["car_flag"] = preprocessed_data["title"].apply(lambda s: contains_dictionary_word(s,car_dictionary))
df["gender_M_flag"] = preprocessed_data["title"].apply(lambda s: contains_dictionary_word(s,male_gender_dictionary))
df["gender_F_flag"] = preprocessed_data["title"].apply(lambda s: contains_dictionary_word(s,female_gender_dictionary))
df["heavy_vehicle_flag"] = preprocessed_data["title"].apply(lambda s: contains_dictionary_word(s,heavy_vehicle_dictionary))
df["pedestrian_flag"] = preprocessed_data["title"].apply(lambda s: contains_dictionary_word(s,pedestrian_dictionary))
df["illegal_flag"] = preprocessed_data["title"].apply(lambda s: contains_dictionary_word(s,drug_alcohol_dictionary))
df["traffic_flag"] = preprocessed_data["title"].apply(lambda s: contains_dictionary_word(s,traffic_dictionary))
df["vehicle_count"] = df[['motorcycle_flag', 'bike_flag', 'car_flag', 'heavy_vehicle_flag']].sum(axis=1)
df["control_flag"] = preprocessed_data["title"].apply(lambda s: contains_dictionary_word(s,control_dictionary))

In [ ]:
data = {'accident_flag': df["accident_flag"],'hospital_flag': df["hospital_flag"],
'fatal_flag': df["fatal_flag"],'injury_flag': df["injury_flag"],'severe_flag': df["severe_flag"],'minor_flag': df["minor_flag"],'motorcycle_flag': df["motorcycle_flag"],
'bike_flag': df["bike_flag"],'car_flag': df["car_flag"],'gender_M_flag': df["gender_M_flag"],'gender_F_flag': df["gender_F_flag"],'heavy_vehicle_flag': df["heavy_vehicle_flag"],
'pedestrian_flag': df["pedestrian_flag"],'illegal_flag': df["illegal_flag"],'traffic_flag': df["traffic_flag"],'control_flag': df["control_flag"]
       ,'vehicle_count': df["vehicle_count"]}
flagged_data = pd.DataFrame(data)
flagged_data

,accident_flag,hospital_flag,fatal_flag,injury_flag,severe_flag,minor_flag,motorcycle_flag,bike_flag,car_flag,gender_M_flag,gender_F_flag,heavy_vehicle_flag,pedestrian_flag,illegal_flag,traffic_flag,control_flag,vehicle_count
0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,2
1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,2
2,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,2
3,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,2
4,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
428,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
429,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
430,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
res2 = list(flagged_data.columns)
joinedlist = res1 + res2

In [ ]:
dfs = [preprocessed_data,flagged_data]

In [ ]:
combined_traffic_data = pd.concat(dfs, axis=1, join='outer', ignore_index=True)
combined_traffic_data.columns = joinedlist

In [ ]:
combined_traffic_data.to_csv('cleaned_data_using_title.csv', index=False)

In [ ]:
cars = sum(flagged_data["car_flag"])
motorcycles = sum(flagged_data["motorcycle_flag"])
bike = sum(flagged_data["bike_flag"])
heavy = sum(flagged_data["heavy_vehicle_flag"])
control = sum(flagged_data["control_flag"])

accidents = sum(flagged_data["accident_flag"])
fatal = sum(flagged_data["fatal_flag"])
serious = sum(flagged_data["severe_flag"])
minor = sum(flagged_data["minor_flag"])

In [ ]:
print("From word extraction we managed to conclude that there were;", "\n",
      "A total of  ", cars, "  car related entries", "\n",   #68
     "A total of  ", motorcycles, "  motorcycle related entries", "\n",  #109
      "A total of  ", bike, "  bike related entries","\n",  #8
      "A total of  ", heavy, "  heavy vehicle related entries", "\n", #29
      "A total of ", control, " loss of control/object related entries", #31
      "\n")

From word extraction we managed to conclude that there were; 
 A total of   68   car related entries 
 A total of   109   motorcycle related entries 
 A total of   8   bike related entries 
 A total of   29   heavy vehicle related entries 
 A total of  31  loss of control/object related entries 



In [ ]:
print("We also managed to extract that there were;", "\n",
     "A total of  ", accidents, "  accidents", "\n", #273
     "A total of  ", fatal, "  fatalities", "\n", #34
     "A total of  ", serious, "  serious injuries", "\n", #219
     "A total of  ", minor, "  minor injuries", "\n") #2

We also managed to extract that there were; 
 A total of   273   accidents 
 A total of   34   fatalities 
 A total of   219   serious injuries 
 A total of   2   minor injuries 



In [ ]:
#Using the content column
df["accident_flag"] = preprocessed_data["content"].apply(lambda s: contains_dictionary_word(s,accident_dictionary))
df["hospital_flag"] = preprocessed_data["content"].apply(lambda s: contains_dictionary_word(s,hospital_dictionary))
df["fatal_flag"] = preprocessed_data["content"].apply(lambda s: contains_dictionary_word(s,fatal_dictionary))
df["injury_flag"] = preprocessed_data["content"].apply(lambda s: contains_dictionary_word(s,injury_dictionary))
df["severe_flag"] = preprocessed_data["content"].apply(lambda s: contains_dictionary_word(s,severe_dictionary))
df["minor_flag"] = preprocessed_data["content"].apply(lambda s: contains_dictionary_word(s,minor_dictionary))
df["motorcycle_flag"] = preprocessed_data["content"].apply(lambda s: contains_dictionary_word(s,motorcycle_dictionary))
df["bike_flag"] = preprocessed_data["content"].apply(lambda s: contains_dictionary_word(s,bike_dictionary))
df["car_flag"] = preprocessed_data["content"].apply(lambda s: contains_dictionary_word(s,car_dictionary))
df["gender_M_flag"] = preprocessed_data["content"].apply(lambda s: contains_dictionary_word(s,male_gender_dictionary))
df["gender_F_flag"] = preprocessed_data["content"].apply(lambda s: contains_dictionary_word(s,female_gender_dictionary))
df["heavy_vehicle_flag"] = preprocessed_data["content"].apply(lambda s: contains_dictionary_word(s,heavy_vehicle_dictionary))
df["pedestrian_flag"] = preprocessed_data["content"].apply(lambda s: contains_dictionary_word(s,pedestrian_dictionary))
df["illegal_flag"] = preprocessed_data["content"].apply(lambda s: contains_dictionary_word(s,drug_alcohol_dictionary))
df["traffic_flag"] = preprocessed_data["content"].apply(lambda s: contains_dictionary_word(s,traffic_dictionary))
df["vehicle_count"] = df[['motorcycle_flag', 'bike_flag', 'car_flag', 'heavy_vehicle_flag']].sum(axis=1)
df["control_flag"] = preprocessed_data["content"].apply(lambda s: contains_dictionary_word(s,control_dictionary))

In [ ]:
data = {'accident_flag': df["accident_flag"],'hospital_flag': df["hospital_flag"],
'fatal_flag': df["fatal_flag"],'injury_flag': df["injury_flag"],'severe_flag': df["severe_flag"],'minor_flag': df["minor_flag"],'motorcycle_flag': df["motorcycle_flag"],
'bike_flag': df["bike_flag"],'car_flag': df["car_flag"],'gender_M_flag': df["gender_M_flag"],'gender_F_flag': df["gender_F_flag"],'heavy_vehicle_flag': df["heavy_vehicle_flag"],
'pedestrian_flag': df["pedestrian_flag"],'illegal_flag': df["illegal_flag"],'traffic_flag': df["traffic_flag"],'control_flag': df["control_flag"]
       ,'vehicle_count': df["vehicle_count"]}
flagged_data = pd.DataFrame(data)
flagged_data

,accident_flag,hospital_flag,fatal_flag,injury_flag,severe_flag,minor_flag,motorcycle_flag,bike_flag,car_flag,gender_M_flag,gender_F_flag,heavy_vehicle_flag,pedestrian_flag,illegal_flag,traffic_flag,control_flag,vehicle_count
0,1,1,0,1,1,0,1,0,1,1,1,0,0,0,1,0,2
1,1,1,0,1,1,0,1,0,1,1,0,0,0,0,1,0,2
2,1,1,0,1,1,0,1,0,1,1,0,0,0,0,1,0,2
3,1,1,0,1,1,0,1,0,1,1,0,0,0,0,1,0,2
4,1,1,0,1,1,0,1,0,1,1,0,0,0,0,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,2
428,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1
429,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1
430,1,1,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0


In [ ]:
dfs = [preprocessed_data,flagged_data]

In [ ]:
combined_traffic_data = pd.concat(dfs, axis=1, join='outer', ignore_index=True)
combined_traffic_data.columns = joinedlist

In [ ]:
combined_traffic_data.to_csv('cleaned_data_using_content.csv', index=False)

In [ ]:
cars = sum(flagged_data["car_flag"])
motorcycles = sum(flagged_data["motorcycle_flag"])
bike = sum(flagged_data["bike_flag"])
heavy = sum(flagged_data["heavy_vehicle_flag"])
control = sum(flagged_data["control_flag"])

accidents = sum(flagged_data["accident_flag"])
fatal = sum(flagged_data["fatal_flag"])
serious = sum(flagged_data["severe_flag"])
minor = sum(flagged_data["minor_flag"])

In [ ]:
print("From word extraction we managed to conclude that there were;", "\n",
      "A total of  ", cars, "  car related entries", "\n", #326
     "A total of  ", motorcycles, "  motorcycle related entries", "\n", #234
      "A total of  ", bike, "  bike related entries","\n", #10
      "A total of  ", heavy, "  heavy vehicle related entries", "\n", #132
      "A total of ", control, " loss of control/object related entries", #208
      "\n")

From word extraction we managed to conclude that there were; 
 A total of   326   car related entries 
 A total of   234   motorcycle related entries 
 A total of   10   bike related entries 
 A total of   132   heavy vehicle related entries 
 A total of  208  loss of control/object related entries 



In [ ]:
print("We also managed to extract that there were;", "\n",
     "A total of  ", accidents, "  accidents", "\n", #375
     "A total of  ", fatal, "  fatalities", "\n", #188
     "A total of  ", serious, "  serious injuries", "\n", #375
     "A total of  ", minor, "  minor injuries", "\n") #49

We also managed to extract that there were; 
 A total of   375   accidents 
 A total of   188   fatalities 
 A total of   375   serious injuries 
 A total of   49   minor injuries 

